1.2 CoefficientFunctions
====

In NGSolve, `CoefficientFunction`s are representations of functions defined on the computational domain. Examples are expressions of coordinate variables $x, y, z$ and functions that are  constant on subdomains. Much of the magic behind the seamless integration of NGSolve with python lies in `CoefficientFunction`s. This tutorial introduces you to them.

After this tutorial you will know how to 

- **define** a `CoefficientFunction`,
- **visualize**  a `CoefficientFunction`,
- **evaluate** `CoefficientFunction`s at points,
- print the **expression tree** of `CoefficientFunction`,
- **integrate** a `CoefficientFunction`, 
- **differentiate** a `CoefficientFunction`, 
- include **parameter** in `CoefficientFunction`s,
- **interpolate** a `CoefficientFunction` into a finite element space, 
- define **vector-valued** `CoefficientFunction`s, and 
- **compile** `CoefficientFunction`s.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
import matplotlib.pyplot as plt
mesh = Mesh (unit_square.GenerateMesh(maxh=0.2))

### Define a function

In [ ]:
myfunc = x*(1-x)
myfunc   # You have just created a CoefficientFunction

In [ ]:
x        # This is a built-in CoefficientFunction

### Visualize the function

Use the `mesh` to visualize the function in Netgen's GUI.

In [ ]:
Draw(myfunc, mesh, "firstfun")

### Evaluate the function

In [ ]:
mip = mesh(0.2, 0.2)
myfunc(mip)

Note that `myfunc(0.2,0.3)` will not evaluate the function: one must give points in the form of `MappedIntegrationPoint`s like `mip` above. The `mesh` knows how to produce them.

### Examining functions on sets of points

In [ ]:
pts = [(0.1*i, 0.2) for i in range(10)]
vals = [myfunc(mesh(*p)) for p in pts] 
for p,v in zip(pts, vals):
    print("point=(%3.2f,%3.2f), value=%6.5f"
         %(p[0], p[1], v))

We may plot the restriction of the `CoefficientFunction` on a line using matplotlib. 

In [ ]:
px = [0.01*i for i in range(100)]
vals = [myfunc(mesh(p,0.5)) for p in px]
plt.plot(px,vals)
plt.xlabel('x')
plt.show()

### Expression tree of a function

Internally, coefficient functions are implemented as an expression tree made from building blocks like `x`, `y`, `sin`, etc., and arithmetic operations.

E.g., the expression tree for `myfunc = x*(1-x)` looks like this:

In [ ]:
print(myfunc) 

### Integrate a function

We can numerically integrate the function using the mesh:

In [ ]:
Integrate(myfunc, mesh)

You can change the precision of the quadrature rule used for the integration using the key word argument `order`. 

### Differentiate a function

Automatic differentiation of a `CoefficientFunction` is now possible through the `Diff` method. Here is how you get $\partial / \partial x$ of `myfunc`:

In [ ]:
diff_myfunc = myfunc.Diff(x)
Draw(diff_myfunc, mesh, "derivative")

See if you can recognize an implementation of the product rule 

$$
\frac{\partial}{\partial x} x (1-x)
= 
\frac{\partial x}{\partial x} 
(1-x) + 
x\frac{\partial (1-x)}{\partial x} 
$$

in the tree-representation of the differentiated coefficient function, printed below.

In [ ]:
print(diff_myfunc)

### Parameters in functions

When building complex coefficient functions from simple ones like `x` and `y`, you may often want to introduce `Parameter`s, which are constants whose value may be changed later.

In [ ]:
k = Parameter(1.0)
f = sin(k*y)
Draw(f, mesh, "f")

The same `f` may be given a different value of `k` later:

In [ ]:
k.Set(10)
Draw(f, mesh, "f")

Look at the expression tree of `f`:

In [ ]:
print(f)

Note how the `Parameter` 
is now a **node** in the expression tree.  You can differentiate a coefficient function with respect to such quantities by passing it as argument to `Diff`:

In [ ]:
f.Diff(k)

In [ ]:
Integrate((f.Diff(k) - y*cos(k*y))**2, mesh)

### Interpolate a function

We may `Set` a `GridFunction` using a `CoefficientFunction`:

In [ ]:
fes = H1(mesh, order=1)
u = GridFunction(fes)
u.Set(myfunc)
Draw(u)         # Cf. Draw(myfunc, mesh, "firstfun")

* The `Set` method interpolates `myfunc` to an element `u` in the finite element space.

* `Set` does an *Oswald-type interpolation* as follows:
    - It first zeros the grid function;
    - It then projects `myfunc` in $L^2$ on each mesh element;
    - It then averages dofs on element interfaces for conformity.
    
* We will see other ways to interpolate in [2.10](../unit-2.10-dualbasis/dualbasis.ipynb).

### Vector-valued `CoefficientFunction`

Here is an example of a vector-valued coefficient function.

In [ ]:
vecfun = CoefficientFunction((-y, sin(x)))
Draw(vecfun, mesh, "vecfun")

Click on `Draw Surface Vectors` in the `Visual` menu to visualize this vector field.

Another example of a vector-valued coefficient function is the gradient of the above-set `GridFunction`. 

In [ ]:
u.Set(myfunc)
gradu = grad(u)
Draw(gradu, mesh, "grad_firstfun")

### Compiled `CoefficientFunction`

Evaluation of a `CoefficientFunction` at a point is usually done 
by traversing its expression tree and evaluating each node of the tree. When the tree has repeated nodes, this is likely wasteful. NGSolve allows you to **"compile"** a `CoefficientFunction` to increase the efficiency of its evaluation. The compilation translates the expression tree into a sequence of linear steps. 

Continuing with our simple `myfunc` example, here is how to use the `Compile` method:

In [ ]:
myfunc_compiled = myfunc.Compile()

Now look at the differences between the compiled and non-compiled `CoefficientFunction`:

In [ ]:
print(myfunc)

In [ ]:
print(myfunc_compiled)

Evaluation of the compiled function is now a linear sequence of Steps 0, 1, 2, and 3 above. We understand the printed description of Steps 2 and 3 above to mean the following.

*  Step 2: (Output of Step 1) - (Output of Step 0) 
*  Step 3: (Output of Step 0) * (Output of Step 2) 

Here is another example, along with differences in timings for integrating the coefficient function on the mesh. 

In [ ]:
f0 = myfunc
f1 = f0*y 
f2 = f1*f1 + f1*f0 + f0*f0 
f3 = f2*f2*f2*f0**2 + f0*f2**2 + f0**2 + f1**2 + f2**2
final = f3 + f3 + f3 
finalc = final.Compile()

In [ ]:
%timeit Integrate(final, mesh, order=10)

In [ ]:
%timeit Integrate(finalc, mesh, order=10)

If your NGSolve installation has a compiler script (you likely do if you built from source using a compiler), then you have the option of letting that compiler optimize your coefficient function further. Here is an example:

In [ ]:
finalcc = final.Compile(realcompile=True, wait=True)

In [ ]:
%timeit Integrate(finalcc, mesh, order=10)

In [ ]:
print(finalc)

In [ ]:
print(final)